# OPEN-CV PROJECT: LIS

# Import all dependencies

In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt

#  Holistic model

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [10]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_landmarks_custom(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             )

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        black = np.zeros((1080, 1920, 3))
        ret, frame = cap.read()
        #frame = cv2.flip(frame,1)
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        points_rh_x = []
        points_rh_y = []
        if results.right_hand_landmarks:
            for i in range(21):
                points_rh_x.append(results.right_hand_landmarks.landmark[i].x)
                points_rh_y.append(results.right_hand_landmarks.landmark[i].y)
                        
        draw_landmarks_custom(frame, results)
        
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Extract point for training and prediction
* Calculate x and y min/max
* Extract points from results
* Normalize between 0 and 1

In [13]:
def points_detection(results):
    
    xMax = max([i.x for i in results.right_hand_landmarks.landmark])
    xMin = min([i.x for i in results.right_hand_landmarks.landmark])
    yMax = max([i.y for i in results.right_hand_landmarks.landmark])
    yMin = min([i.y for i in results.right_hand_landmarks.landmark])
    
    rh = np.array([[points.x, points.y, points.z] for points in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    for i in np.arange(0, 63, 3):
        rh[i]=(rh[i]-xMin)/(xMax-xMin)
    for i in np.arange(1, 63, 3):
        rh[i]=(rh[i]-yMin)/(yMax-yMin)

    # lh = np.array([[points.x, points.y, points.z] for points in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    # po = np.array([[points.x, points.y, points.z] for points in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(99)
    # return np.concatenate([lh, rh, po])

    return rh

# Setup Folders for dataset

In [14]:
DATA_PATH = os.path.join('LIS_data')

# Set number of output and number of sample for each label

In [15]:
labels = np.array(['a', 'b', 'c']) # put the entire alphabet in the future
no_sequences = 101

# Collect data for training

In [44]:
data = []
#no_sequences = no_sequences+1

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for label in labels:
        for id in range(no_sequences):
    
            ret, frame = cap.read()

            # make detection
            image, results = mediapipe_detection(frame, holistic)

            draw_landmarks_custom(frame, results)

            if id == 0:
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                #cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                cv2.imshow('LIS', frame)
                cv2.waitKey(1000)
            else:                
                data.append(points_detection(results))
                cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA) 
                cv2.waitKey(100)
                cv2.imshow('LIS', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Save dataset for training

In [45]:
import pandas as pd
df = pd.DataFrame(np.array(data))
y=[]
for i in labels:
    y = np.concatenate([y, [i] * (no_sequences-1)])
df['y'] = y
pd.DataFrame(df).to_csv('data_rh.csv')
print(df['y'].value_counts())
df

a    100
b    100
c    100
Name: y, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,y
0,0.425902,1.0,1.340059e-07,0.695783,0.874607,-0.020425,0.901603,0.706403,-0.027040,0.963582,...,0.080884,0.412132,-0.021941,0.038354,0.317079,-0.027061,0.000000,0.220971,-0.030792,a
1,0.390425,1.0,2.664214e-07,0.658640,0.885224,-0.022115,0.871776,0.730142,-0.030296,0.959132,...,0.072933,0.423192,-0.022581,0.034754,0.327254,-0.028437,0.000000,0.229601,-0.032413,a
2,0.374288,1.0,2.821520e-07,0.634966,0.894772,-0.021671,0.841212,0.736666,-0.029467,0.930800,...,0.057700,0.429490,-0.024197,0.024144,0.335471,-0.029346,0.000000,0.239120,-0.032613,a
3,0.387199,1.0,2.822724e-07,0.639301,0.889030,-0.022174,0.836478,0.730278,-0.031065,0.921737,...,0.065610,0.424691,-0.025182,0.029768,0.332378,-0.030258,0.000000,0.238768,-0.033651,a
4,0.365346,1.0,3.105052e-07,0.622440,0.896614,-0.020286,0.829070,0.741293,-0.028262,0.923729,...,0.066284,0.426324,-0.026726,0.029319,0.332388,-0.031293,0.000000,0.236391,-0.034014,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.361399,1.0,4.446333e-07,0.681673,0.900372,-0.022844,0.768471,0.705740,-0.029622,0.502139,...,0.000000,0.558536,-0.041691,0.114536,0.669546,-0.041196,0.176285,0.747251,-0.033475,c
296,0.366428,1.0,4.880291e-07,0.632464,0.904738,-0.026527,0.702697,0.716121,-0.035356,0.448560,...,0.000000,0.571557,-0.045612,0.106128,0.681197,-0.047473,0.178052,0.747528,-0.041341,c
297,0.363903,1.0,4.404679e-07,0.655993,0.894866,-0.023968,0.725929,0.700309,-0.030566,0.451680,...,0.000000,0.557114,-0.043115,0.125017,0.663191,-0.044855,0.220544,0.736120,-0.038749,c
298,0.342652,1.0,4.077550e-07,0.587155,0.884288,-0.023020,0.648971,0.691969,-0.029894,0.410349,...,0.000000,0.552957,-0.041557,0.116282,0.658739,-0.042782,0.208386,0.731682,-0.036044,c


# Model

In [46]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler
import datetime
from datetime import datetime

Using TensorFlow backend.


In [48]:
label_map = {label:num for num, label in enumerate(labels)}
label_map

{'a': 0, 'b': 1, 'c': 2}

In [49]:
# split into input (X) and output (y) variables
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1])

# define min max scaler
scaler = MinMaxScaler()

# transform data
X = scaler.fit_transform(X)

# define LabelEncoder
label_encoder = LabelEncoder()

# transform data
integer_encoded = label_encoder.fit_transform(y)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

In [50]:
# define the keras model
model = Sequential()
model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(labels.shape[0], activation='softmax'))

2021-11-23 16:04:24.128162: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-23 16:04:24.209071: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd01872f2f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-23 16:04:24.209107: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [51]:
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [52]:
# fit the keras model on the dataset
model.fit(X, y, epochs=500, batch_size=20, validation_split=0.2)

Train on 240 samples, validate on 60 samples
Epoch 1/500
240/240 [==============================] - 1s 2ms/step - loss: 0.9779 - categorical_accuracy: 0.5625 - val_loss: 0.7931 - val_categorical_accuracy: 1.0000
Epoch 2/500
240/240 [==============================] - 0s 128us/step - loss: 0.6430 - categorical_accuracy: 0.9542 - val_loss: 0.7618 - val_categorical_accuracy: 0.0667
Epoch 3/500
240/240 [==============================] - 0s 269us/step - loss: 0.3538 - categorical_accuracy: 0.8500 - val_loss: 0.9594 - val_categorical_accuracy: 0.0000e+00
Epoch 4/500
240/240 [==============================] - 0s 280us/step - loss: 0.2255 - categorical_accuracy: 0.9167 - val_loss: 0.5931 - val_categorical_accuracy: 0.9000
Epoch 5/500
240/240 [==============================] - 0s 212us/step - loss: 0.1015 - categorical_accuracy: 1.0000 - val_loss: 0.2037 - val_categorical_accuracy: 1.0000
Epoch 6/500
240/240 [==============================] - 0s 270us/step - loss: 0.0276 - categorical_accuracy: 

240/240 [==============================] - 0s 142us/step - loss: 6.3574e-05 - categorical_accuracy: 1.0000 - val_loss: 8.2271e-04 - val_categorical_accuracy: 1.0000
Epoch 49/500
240/240 [==============================] - 0s 167us/step - loss: 6.0784e-05 - categorical_accuracy: 1.0000 - val_loss: 7.9235e-04 - val_categorical_accuracy: 1.0000
Epoch 50/500
240/240 [==============================] - 0s 200us/step - loss: 5.8235e-05 - categorical_accuracy: 1.0000 - val_loss: 7.8066e-04 - val_categorical_accuracy: 1.0000
Epoch 51/500
240/240 [==============================] - 0s 142us/step - loss: 5.6025e-05 - categorical_accuracy: 1.0000 - val_loss: 7.6693e-04 - val_categorical_accuracy: 1.0000
Epoch 52/500
240/240 [==============================] - 0s 206us/step - loss: 5.3654e-05 - categorical_accuracy: 1.0000 - val_loss: 7.2283e-04 - val_categorical_accuracy: 1.0000
Epoch 53/500
240/240 [==============================] - 0s 112us/step - loss: 5.1687e-05 - categorical_accuracy: 1.0000 - v

Epoch 140/500
240/240 [==============================] - 0s 74us/step - loss: 3.0755e-06 - categorical_accuracy: 1.0000 - val_loss: 3.5299e-05 - val_categorical_accuracy: 1.0000
Epoch 141/500
240/240 [==============================] - 0s 78us/step - loss: 3.0239e-06 - categorical_accuracy: 1.0000 - val_loss: 3.4821e-05 - val_categorical_accuracy: 1.0000
Epoch 142/500
240/240 [==============================] - 0s 76us/step - loss: 2.9683e-06 - categorical_accuracy: 1.0000 - val_loss: 3.4578e-05 - val_categorical_accuracy: 1.0000
Epoch 143/500
240/240 [==============================] - 0s 96us/step - loss: 2.9206e-06 - categorical_accuracy: 1.0000 - val_loss: 3.4062e-05 - val_categorical_accuracy: 1.0000
Epoch 144/500
240/240 [==============================] - 0s 107us/step - loss: 2.8744e-06 - categorical_accuracy: 1.0000 - val_loss: 3.3820e-05 - val_categorical_accuracy: 1.0000
Epoch 145/500
240/240 [==============================] - 0s 124us/step - loss: 2.8391e-06 - categorical_accur

Epoch 186/500
240/240 [==============================] - 0s 106us/step - loss: 1.5596e-06 - categorical_accuracy: 1.0000 - val_loss: 2.3002e-05 - val_categorical_accuracy: 1.0000
Epoch 187/500
240/240 [==============================] - 0s 133us/step - loss: 1.5368e-06 - categorical_accuracy: 1.0000 - val_loss: 2.2816e-05 - val_categorical_accuracy: 1.0000
Epoch 188/500
240/240 [==============================] - 0s 106us/step - loss: 1.5115e-06 - categorical_accuracy: 1.0000 - val_loss: 2.2462e-05 - val_categorical_accuracy: 1.0000
Epoch 189/500
240/240 [==============================] - 0s 123us/step - loss: 1.4906e-06 - categorical_accuracy: 1.0000 - val_loss: 2.2319e-05 - val_categorical_accuracy: 1.0000
Epoch 190/500
240/240 [==============================] - 0s 133us/step - loss: 1.4727e-06 - categorical_accuracy: 1.0000 - val_loss: 2.2146e-05 - val_categorical_accuracy: 1.0000
Epoch 191/500
240/240 [==============================] - 0s 154us/step - loss: 1.4533e-06 - categorical_a

Epoch 232/500
240/240 [==============================] - 0s 149us/step - loss: 8.9704e-07 - categorical_accuracy: 1.0000 - val_loss: 1.6798e-05 - val_categorical_accuracy: 1.0000
Epoch 233/500
240/240 [==============================] - 0s 184us/step - loss: 8.8661e-07 - categorical_accuracy: 1.0000 - val_loss: 1.6673e-05 - val_categorical_accuracy: 1.0000
Epoch 234/500
240/240 [==============================] - 0s 121us/step - loss: 8.7469e-07 - categorical_accuracy: 1.0000 - val_loss: 1.6541e-05 - val_categorical_accuracy: 1.0000
Epoch 235/500
240/240 [==============================] - 0s 116us/step - loss: 8.6625e-07 - categorical_accuracy: 1.0000 - val_loss: 1.6534e-05 - val_categorical_accuracy: 1.0000
Epoch 236/500
240/240 [==============================] - 0s 120us/step - loss: 8.5929e-07 - categorical_accuracy: 1.0000 - val_loss: 1.6400e-05 - val_categorical_accuracy: 1.0000
Epoch 237/500
240/240 [==============================] - 0s 109us/step - loss: 8.4986e-07 - categorical_a

Epoch 278/500
240/240 [==============================] - 0s 130us/step - loss: 5.5233e-07 - categorical_accuracy: 1.0000 - val_loss: 1.2994e-05 - val_categorical_accuracy: 1.0000
Epoch 279/500
240/240 [==============================] - 0s 122us/step - loss: 5.4687e-07 - categorical_accuracy: 1.0000 - val_loss: 1.3002e-05 - val_categorical_accuracy: 1.0000
Epoch 280/500
240/240 [==============================] - 0s 124us/step - loss: 5.3992e-07 - categorical_accuracy: 1.0000 - val_loss: 1.2926e-05 - val_categorical_accuracy: 1.0000
Epoch 281/500
240/240 [==============================] - 0s 128us/step - loss: 5.3445e-07 - categorical_accuracy: 1.0000 - val_loss: 1.2827e-05 - val_categorical_accuracy: 1.0000
Epoch 282/500
240/240 [==============================] - 0s 145us/step - loss: 5.2899e-07 - categorical_accuracy: 1.0000 - val_loss: 1.2774e-05 - val_categorical_accuracy: 1.0000
Epoch 283/500
240/240 [==============================] - 0s 137us/step - loss: 5.2253e-07 - categorical_a

Epoch 324/500
240/240 [==============================] - 0s 169us/step - loss: 3.5812e-07 - categorical_accuracy: 1.0000 - val_loss: 1.0545e-05 - val_categorical_accuracy: 1.0000
Epoch 325/500
240/240 [==============================] - 0s 247us/step - loss: 3.5365e-07 - categorical_accuracy: 1.0000 - val_loss: 1.0479e-05 - val_categorical_accuracy: 1.0000
Epoch 326/500
240/240 [==============================] - 0s 163us/step - loss: 3.4918e-07 - categorical_accuracy: 1.0000 - val_loss: 1.0426e-05 - val_categorical_accuracy: 1.0000
Epoch 327/500
240/240 [==============================] - 0s 147us/step - loss: 3.4471e-07 - categorical_accuracy: 1.0000 - val_loss: 1.0485e-05 - val_categorical_accuracy: 1.0000
Epoch 328/500
240/240 [==============================] - 0s 169us/step - loss: 3.4173e-07 - categorical_accuracy: 1.0000 - val_loss: 1.0501e-05 - val_categorical_accuracy: 1.0000
Epoch 329/500
240/240 [==============================] - 0s 175us/step - loss: 3.3776e-07 - categorical_a

Epoch 370/500
240/240 [==============================] - 0s 105us/step - loss: 2.3196e-07 - categorical_accuracy: 1.0000 - val_loss: 8.8067e-06 - val_categorical_accuracy: 1.0000
Epoch 371/500
240/240 [==============================] - 0s 113us/step - loss: 2.2898e-07 - categorical_accuracy: 1.0000 - val_loss: 8.7530e-06 - val_categorical_accuracy: 1.0000
Epoch 372/500
240/240 [==============================] - 0s 129us/step - loss: 2.2799e-07 - categorical_accuracy: 1.0000 - val_loss: 8.7610e-06 - val_categorical_accuracy: 1.0000
Epoch 373/500
240/240 [==============================] - 0s 121us/step - loss: 2.2600e-07 - categorical_accuracy: 1.0000 - val_loss: 8.7312e-06 - val_categorical_accuracy: 1.0000
Epoch 374/500
240/240 [==============================] - 0s 120us/step - loss: 2.2302e-07 - categorical_accuracy: 1.0000 - val_loss: 8.6100e-06 - val_categorical_accuracy: 1.0000
Epoch 375/500
240/240 [==============================] - 0s 126us/step - loss: 2.2203e-07 - categorical_a

Epoch 416/500
240/240 [==============================] - 0s 132us/step - loss: 1.5646e-07 - categorical_accuracy: 1.0000 - val_loss: 7.4241e-06 - val_categorical_accuracy: 1.0000
Epoch 417/500
240/240 [==============================] - 0s 127us/step - loss: 1.5547e-07 - categorical_accuracy: 1.0000 - val_loss: 7.4201e-06 - val_categorical_accuracy: 1.0000
Epoch 418/500
240/240 [==============================] - 0s 126us/step - loss: 1.5398e-07 - categorical_accuracy: 1.0000 - val_loss: 7.4062e-06 - val_categorical_accuracy: 1.0000
Epoch 419/500
240/240 [==============================] - 0s 118us/step - loss: 1.5249e-07 - categorical_accuracy: 1.0000 - val_loss: 7.3824e-06 - val_categorical_accuracy: 1.0000
Epoch 420/500
240/240 [==============================] - 0s 130us/step - loss: 1.5199e-07 - categorical_accuracy: 1.0000 - val_loss: 7.3665e-06 - val_categorical_accuracy: 1.0000
Epoch 421/500
240/240 [==============================] - 0s 99us/step - loss: 1.5050e-07 - categorical_ac

Epoch 462/500
240/240 [==============================] - 0s 129us/step - loss: 1.0431e-07 - categorical_accuracy: 1.0000 - val_loss: 6.4169e-06 - val_categorical_accuracy: 1.0000
Epoch 463/500
240/240 [==============================] - 0s 113us/step - loss: 1.0282e-07 - categorical_accuracy: 1.0000 - val_loss: 6.4388e-06 - val_categorical_accuracy: 1.0000
Epoch 464/500
240/240 [==============================] - 0s 107us/step - loss: 1.0232e-07 - categorical_accuracy: 1.0000 - val_loss: 6.3732e-06 - val_categorical_accuracy: 1.0000
Epoch 465/500
240/240 [==============================] - 0s 119us/step - loss: 1.0033e-07 - categorical_accuracy: 1.0000 - val_loss: 6.3911e-06 - val_categorical_accuracy: 1.0000
Epoch 466/500
240/240 [==============================] - 0s 143us/step - loss: 9.9838e-08 - categorical_accuracy: 1.0000 - val_loss: 6.3871e-06 - val_categorical_accuracy: 1.0000
Epoch 467/500
240/240 [==============================] - 0s 148us/step - loss: 9.8844e-08 - categorical_a

# Save models

In [54]:
# serialize model to JSON
model_json = model.to_json()
with open("model_rh.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_rh.h5")
print("Saved model to disk")

Saved model to disk


# Evaluate model

In [55]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [56]:
yhat = model.predict(X)
ytrue = np.argmax(y, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [57]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]]])

In [58]:
accuracy_score(ytrue, yhat)

1.0

# Testing

In [59]:
from keras.models import model_from_json

# load json and create model
json_file = open('model_rh.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_rh.h5")
print("Loaded model from disk")

Loaded model from disk


# Set parameters for prediction

In [60]:
threshold = 0.4
min_detection_confidence = 0.5
min_tracking_confidence = 0.5

# Real time detection and prediction

In [ ]:
cap = cv2.VideoCapture(0)


with mp_holistic.Holistic(min_detection_confidence=min_detection_confidence, 
                          min_tracking_confidence=min_tracking_confidence) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w, c = frame.shape

        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        
        cv2.rectangle(frame, (0+int(0.03*h),int(h-0.14*h)), (0+int(0.75*h), int(h-0.015*h)), color,-1)
                         
            
        for i in range(len(labels)):
            cv2.rectangle(frame, (70, 10+ i*int(50)), (70+0, 60+ i*int(50)), color,-1)
            cv2.putText(frame, labels[i], (10, 50+ i*int(50)), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,255,0), 4, cv2.LINE_AA)

        # perform prediction with relative probability
        if results.right_hand_landmarks:
            for i in range(len(labels)):
                cv2.rectangle(frame, (70, 10+ i*int(50)), (70+int(model.predict(np.array([points_detection(results)]))[0][i]*100)*3, 60+ i*int(50)), color,-1)

                cv2.putText(frame, labels[i], (10, 50+ i*int(50)), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,255,0), 4, cv2.LINE_AA)

            prediction = labels[np.argmax(model.predict(np.array([points_detection(results)])))]
            pred_prob = int(np.max(model.predict(np.array([points_detection(results)])))*100)

            # add text with prediction
            if pred_prob > int(threshold*100):
                cv2.putText(frame, f'{prediction.capitalize()} ({pred_prob}%)', 
                            (0+int(0.05*h),h-int(0.05*h)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2 ,
                            (0,255,0), 
                            4, 
                            cv2.LINE_AA)
            else:
                cv2.putText(frame, 'Detecting Hand', 
                            (0+int(0.05*h),h-int(0.05*h)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2 ,
                            (0,255,0), 
                            4, 
                            cv2.LINE_AA)
        
        else:
                cv2.putText(frame, 'Detecting Hand', 
                            (0+int(0.05*h),h-int(0.05*h)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2,
                            (0,255,0), 
                            4, 
                            cv2.LINE_AA)

            
        #draw_landmarks_custom(frame, results)
  
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [153]:
model.predict(np.array([points_detection(results)]))[0][1]

AttributeError: 'NoneType' object has no attribute 'landmark'